In [12]:
crossing_lane_map = {
        1:{
            'left':{
                'in_cross':["78","77","76"],
                'out_cross':['79','80'],
            },
            'down':{
                'in_cross':['37','36','35'],
                'out_cross':['33','34'],
            },
            'right':{
                'in_cross':['85','84','83'],
                'out_cross':['81','82']
            }
        },
        2:{
            'left':{
                'in_cross':["90","89","88"],
                'out_cross':['91','92'],
            },
            'down':{
                'in_cross':['235','236','237','238'],
                'out_cross':['239','240','241'],
            },
            'right':{
                'in_cross':['97','96','95'],
                'out_cross':['93','94']
            }
        },
        3:{
            'left':{
                'in_cross':["102","101","100"],
                'out_cross':['103','104'],
            },
            'down':{
                'in_cross':['219','328','327'],
                'out_cross':['326','208'],
            },
            'right':{
                'in_cross':['109','108','107'],
                'out_cross':['105','106']
            }
        },
        4:{
            'left':{
                'in_cross':["114","113","112"],
                'out_cross':['115','116'],
            },
            'down':{
                'in_cross':['228','229','230','231'],
                'out_cross':['232','233','234','223'],
            },
            'right':{
                'in_cross':['121','120','119'],
                'out_cross':['117','118']
            }
        },
        5:{
            'left':{
                'in_cross':["205","191","190",'189','188','187'],
                'out_cross':['192','193','194','195','196','206'],
            },
            'down':{
                'in_cross':['71','70','69'],
                'out_cross':['67','68'],
            },
            'right':{
                'in_cross':['203','202','201','200'],
                'out_cross':['199','198','197'],
            },
            'up':{
                'in_cross':['64','63','62'],
                'out_cross':['65','66'],
            }
        },
        6:{
            'left':{
                'in_cross':["262","261","260",'259'],
                'out_cross':['264','265'],
            },
            'down':{
                'in_cross':['325','324','323','322'],
                'out_cross':['319','320','321'],
            },
            'right':{
                'in_cross':['284','283','282','281'],
                'out_cross':['272','273','274','275','276'],
            },
            'up':{
                'in_cross':['303','302','301','300'],
                'out_cross':['304','305','306'],
            }
        },
        7:{
            'left':{
                'in_cross':["137","136","135",'134','133'],
                'out_cross':['129','130','131','132'],
            },
            'down':{
                'in_cross':['225','5','4','3'],
                'out_cross':['339','340','220'],
            },
            'right':{
                'in_cross':['157','156','155','154','153'],
                'out_cross':['138','139','140'],
            },
            'up':{
                'in_cross':['213','336','335'],
                'out_cross':['337','338','214'],
            }
        },
        8:{
            'left':{
                'in_cross':["147","146","145",'144','143'],
                'out_cross':['148','149','150'],
            },
            'down':{
                'in_cross':['170','169','168','167'],
                'out_cross':['164','165','166'],
            },
            'right':{
                'in_cross':['184','183','182','181','180'],
                'out_cross':['171','172','173','174'],
            },
            'up':{
                'in_cross':['160','159','158'],
                'out_cross':['161','162','163'],
            }
        },
        9:{
            'down':{
                'in_cross':['20','19','18'],
                'out_cross':['12','13','14'],
            },
            'up':{
                'in_cross':['225','26','25','24'],
                'out_cross':['27','28','29','226'],
            }
        },
    }

In [2]:
import json
with open('lane_clean\lane_v3.geojson','r') as f:
    content = f.read()
    lane_data= json.loads(content)

In [5]:
lane_id_map = {i:d['properties']['fid'] for (i,d) in  enumerate(lane_data['features'])}

In [13]:
for cross in crossing_lane_map.keys():
    for direction in crossing_lane_map[cross].keys():
        for state in crossing_lane_map[cross][direction].keys():
            temp = [ str(lane_id_map[int(index)]) for index in crossing_lane_map[cross][direction][state]]
            crossing_lane_map[cross][direction][state] = temp

In [36]:
with open('data_jam_10.json','r') as f:
    content = f.read()
    jam_data= json.loads(content)

In [37]:
import copy
cross_lane_data = copy.deepcopy(crossing_lane_map)

for cross in crossing_lane_map.keys():
    for direction in crossing_lane_map[cross].keys():
        for state in crossing_lane_map[cross][direction].keys():
            data = dict()
            state = 'in_cross'
            for fid  in crossing_lane_map[cross][direction][state]:
                temp = jam_data[fid]
                for k in temp.keys():
                    if k in data.keys():
                        data[k]['v'] = (data[k]['v']*data[k]['num'] + temp[k]['num'] * temp[k]['v'])/(data[k]['num'] + temp[k]['num'])
                        data[k]['num']  = data[k]['num'] + temp[k]['num']
                        data[k]['stop_num']  = data[k]['stop_num'] + temp[k]['stop_num']
                    else:
                        data[k] = temp[k]
            cross_lane_data[cross][direction] = data
                        

In [39]:
for cross in cross_lane_data.keys():
    for direction in cross_lane_data[cross].keys():
        q = [[0,0] for i in range(60)]
        v_sum = 0
        n_sum = 0
        stop_sum = 0
        for (time,value) in sorted(cross_lane_data[cross][direction].items(),key=lambda x:int(x[0])):
            l_v,l_n = q.pop(0)
            q.append([value['v'],value['num']])
            v_sum = v_sum - l_v*l_n + value['v']*value['num']
            n_sum = n_sum - l_n + value['num']
            stop_sum  = stop_sum + cross_lane_data[cross][direction][time]['stop_num']
            cross_lane_data[cross][direction][time]['v'] = v_sum/n_sum
            cross_lane_data[cross][direction][time]['num'] = n_sum
            cross_lane_data[cross][direction][time]['stop_num'] = stop_sum


In [ ]:
cross_lane_data

In [41]:
del cross_lane_data[9]

In [29]:
cross_lane_data.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8])

In [42]:
with open("data_jam_10_processed.json", "w") as f:
    json.dump(cross_lane_data, f, ensure_ascii=False)

In [25]:
cross_lane_data

{1: {'left': {'1681340400': {'num': 1, 'v': 0.0, 'stop_num': 1},
   '1681340410': {'num': 4, 'v': 5.375696, 'stop_num': 1},
   '1681340420': {'num': 7, 'v': 6.294204857142857, 'stop_num': 0},
   '1681340600': {'num': 31, 'v': 5.000684338709679, 'stop_num': 4},
   '1681340610': {'num': 37, 'v': 4.231053843243244, 'stop_num': 5},
   '1681340620': {'num': 42, 'v': 3.727356957142858, 'stop_num': 5},
   '1681340630': {'num': 47, 'v': 3.330829621276597, 'stop_num': 5},
   '1681340640': {'num': 52, 'v': 3.0105575423076933, 'stop_num': 5},
   '1681340650': {'num': 59, 'v': 2.653372749152543, 'stop_num': 7},
   '1681340660': {'num': 66, 'v': 2.912338848484849, 'stop_num': 1},
   '1681340670': {'num': 67, 'v': 2.8688711044776127, 'stop_num': 1},
   '1681340680': {'num': 68, 'v': 2.8266818235294124, 'stop_num': 1},
   '1681340690': {'num': 69, 'v': 2.7857154202898555, 'stop_num': 1},
   '1681340700': {'num': 72, 'v': 3.049758194444445, 'stop_num': 1},
   '1681340710': {'num': 73, 'v': 3.007980684

In [14]:
crossing_lane_map

{1: {'left': {'in_cross': ['202', '201', '200'], 'out_cross': ['203', '204']},
  'down': {'in_cross': ['105', '104', '103'], 'out_cross': ['101', '102']},
  'right': {'in_cross': ['209', '208', '207'], 'out_cross': ['205', '206']}},
 2: {'left': {'in_cross': ['218', '217', '216'], 'out_cross': ['219', '220']},
  'down': {'in_cross': ['80688', '80689', '80690', '80691'],
   'out_cross': ['80692', '80693', '80694']},
  'right': {'in_cross': ['225', '224', '223'], 'out_cross': ['221', '222']}},
 3: {'left': {'in_cross': ['234', '233', '232'], 'out_cross': ['235', '236']},
  'down': {'in_cross': ['70369', '1003', '1002'],
   'out_cross': ['1001', '70358']},
  'right': {'in_cross': ['241', '240', '239'], 'out_cross': ['237', '238']}},
 4: {'left': {'in_cross': ['250', '249', '248'], 'out_cross': ['251', '252']},
  'down': {'in_cross': ['70416', '80679', '80680', '80681'],
   'out_cross': ['80682', '80683', '80684', '70411']},
  'right': {'in_cross': ['257', '256', '255'], 'out_cross': ['253

In [6]:
lane_id_map

{0: 1037,
 1: 1038,
 2: 1039,
 3: 1040,
 4: 1041,
 5: 1042,
 6: 3105,
 7: 3106,
 8: 3124,
 9: 3125,
 10: 3126,
 11: 3127,
 12: 3128,
 13: 3129,
 14: 3130,
 15: 3131,
 16: 3132,
 17: 3133,
 18: 3134,
 19: 3135,
 20: 3136,
 21: 3137,
 22: 3138,
 23: 3139,
 24: 3140,
 25: 3141,
 26: 3142,
 27: 3143,
 28: 3144,
 29: 3145,
 30: 3146,
 31: 3147,
 32: 3148,
 33: 101,
 34: 102,
 35: 103,
 36: 104,
 37: 105,
 38: 106,
 39: 107,
 40: 108,
 41: 109,
 42: 110,
 43: 111,
 44: 112,
 45: 113,
 46: 116,
 47: 117,
 48: 118,
 49: 119,
 50: 120,
 51: 121,
 52: 122,
 53: 123,
 54: 126,
 55: 127,
 56: 128,
 57: 129,
 58: 130,
 59: 131,
 60: 132,
 61: 133,
 62: 134,
 63: 135,
 64: 136,
 65: 137,
 66: 138,
 67: 139,
 68: 140,
 69: 155,
 70: 156,
 71: 157,
 72: 163,
 73: 164,
 74: 165,
 75: 166,
 76: 200,
 77: 201,
 78: 202,
 79: 203,
 80: 204,
 81: 205,
 82: 206,
 83: 207,
 84: 208,
 85: 209,
 86: 212,
 87: 215,
 88: 216,
 89: 217,
 90: 218,
 91: 219,
 92: 220,
 93: 221,
 94: 222,
 95: 223,
 96: 224,
 97: 22